In [1]:
import tensorflow as tf
import numpy as np
import keras
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# from: https://towardsdatascience.com/intuition-behind-residual-neural-networks-fa5d2996b2c7

import math
import tensorflow as tf

def sin():
  while True:
    x = tf.random.normal((32, 10), math.pi, 1.5)
    y = tf.math.sin(x)
    yield (x, y)

def build_model(layers):
  x = inputs = tf.keras.Input((10, ))
  for _ in range(layers):
    x = tf.keras.layers.Dense(32, activation="relu")(x)
  x = tf.keras.layers.Dense(10)(x)
  model = tf.keras.Model(inputs, x)
  model.compile("adam", "mse")
  return model

dts = tf.data.Dataset.from_generator(sin, (tf.float32, tf.float32), ((32, 10), (32, 10)))

for t in range(1, 31):
  plain = build_model(t)
  hist = plain.fit(dts, steps_per_epoch=20, epochs=20*t, verbose=0)
  print(hist.history["loss"][-1])

2024-10-27 20:58:56.872508: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2024-10-27 20:58:56.872544: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-10-27 20:58:56.872553: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-10-27 20:58:56.872571: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-27 20:58:56.872587: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-10-27 20:58:57.140124: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


0.11573173105716705
0.02728487178683281
5.7672613351655855e-09
2.9586735763587058e-05
4.097856617590878e-06
1.6608477380941622e-05
0.0002692705311346799
1.397223314825169e-07
0.00013103218225296587
0.0006427859771065414
2.7592193418968236e-08
0.0001288317725993693
6.200579809956253e-05
5.381722303354763e-07


In [ ]:


def resblock(inputs):
  # Asumption: inputs is of last dimension 32.
  x = tf.keras.layers.Dense(32, activation="relu")(inputs)
  x = tf.keras.layers.Dense(32)(x)
  x = x + inputs # Skip connection
  x = tf.keras.Activation("relu")(x)
  return x

def build_res_model(layers):
  x = inputs = tf.keras.Input((10, ))
  for _ in range(layers):
    x = resblock(x)
  x = tf.keras.layers.Dense(10)(x)
  model = tf.keras.Model(inputs, x)
  model.compile("adam", "mse")
  return model

dts = tf.data.Dataset.from_generator(sin, (tf.float32, tf.float32), ((32, 10), (32, 10)))

for t in range(1, 31):
  plain = build_res_model(t)
  hist = plain.fit(dts, steps_per_epoch=20, epochs=20*t, verbose=0)
  print(hist.history["loss"][-1])